### Model assessment: LOO-CV for factory data with Stan (6p)

Use leave-one-out cross validation (LOO-CV) to assess the predictive performance of the pooled, separate and hierarchical Gaussian models for the factory dataset (see the second exercise in Assignment 5). Use Stan for fitting the models, and the provided PSIS-LOO (Pareto smoothed importance sampling LOO) code for computing the approximate LOO-CV given the posterior samples provided by Stan. Your results should include:

 * PSIS-LOO values, the effective number of parameters peff, and the k-values for each of the three models
 * an assessment of whether there are differences between the models, and if so, which model should be selected according to PSIS-LOO
 
Remember also to comment on the results. Hints and further advice:

 * In all the three models, use uniform priors for all the parameters to standardize the model assessment results, making the grading easier.
 * In the hierarchical model, use the same measurement deviation σ for all the machines. This is reasonable, since there are so few measurements per machine, so learning the deviations separately for each machine is difficult. If you want, you can also compute the PSIS-LOO value for the hierarchical model with different σj for each machine and see the effect in the estimated predictive performance (this is not, however, required for full points). For the separate model, you should still use different σj for each machine.
 * In order to use the psisloo-function, you need to compute the log-likelihood values for each observation and for all the posterior samples. This can be done in the generated quantities block; for a demonstration, see the Gaussian linear model for Kilpisjärvi data in the Matlab Stan examples.
 * It is reasonable to visualize the k-values for each model, so that you can easily see how many of these values fall in the range 0.5 < k < 1 or k > 1 to assess the reliability of the PSIS-LOO estimate for each of the models.
 * The estimated effective number of parameters in the model can be computed from equation (7.15) in the book, where lppdloo-cv is the PSIS-LOO value (first output argument of psisloo) and lppd is given by equation (7.5) in the book.
 * PSIS-LOO is a recently developed method for approxmating the exact LOO and is thus not in BDA3. For more information, see the lecture slides and the original paper by Vehtari, Gelman and Gabry (2015) on arXiv http://arxiv.org/pdf/1507.04544v2

In [10]:
%matplotlib inline
import matplotlib.pyplot as plt
from psis import psisloo
import numpy as np
import pystan

plt.rc('font', size=16)
plt.rc('lines', color='#377eb8', linewidth=2)
plt.set_cmap(plt.get_cmap('viridis'));

y = np.loadtxt('../ex5/factory.txt')

In [25]:
data_separate = {
    'J': y.shape[0],
    'K': y.shape[1],
    'y': y
}
code_separate = """
data {
    int<lower=1> J; // number of data points
    int<lower=1> K; // number of groups 
    matrix[J,K]  y; // measurements
}
parameters {
    vector[K] mu; // mean
    vector<lower=0>[K] sigma; // std
}
model {
    for (k in 1:K) {
      y[:,k] ~ normal(mu[k], sigma[k]);
    }
}
generated quantities {
    vector[K] log_like;

    for (k in 1:K)
      log_like[k] <- normal_log(y[:,k], mu[k], sigma[k]);
}
"""
fit_separate = pystan.stan(model_code=code_separate, data=data_separate, iter=3000, chains=20)
params_separate = fit_separate.extract()
print fit_separate

Inference for Stan model: anon_model_b5c8b60b115bf8ed54380ca3ed027dea.
20 chains, each with iter=3000; warmup=1500; thin=1; 
post-warmup draws per chain=1500, total post-warmup draws=30000.

              mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
mu[0]        75.39     0.8  24.24  43.01  68.21  76.05  83.51 108.88  916.0   1.02
mu[1]       106.22    0.14   9.68  87.45  101.6 106.17 110.86 125.28 4707.0    1.0
mu[2]        87.78    0.17  11.01  65.96  82.81   87.8  92.68 108.57 4014.0    1.0
mu[3]       111.48    0.09   6.31  99.03 108.48 111.52 114.49 123.57 4514.0    1.0
mu[4]        90.06    0.15   9.22  72.14   85.8  90.05  94.43 108.01 3975.0    1.0
mu[5]        86.69    0.46  18.72  55.84  78.74  86.17  93.74 119.03 1676.0   1.01
sigma[0]     32.31    0.98   31.0  12.88  19.45  25.68  35.77  87.24 1003.0   1.02
sigma[1]     18.59    0.19  12.52   7.77  11.58  15.34  21.24  50.26 4519.0    1.0
sigma[2]     20.35    0.24  14.64   8.36  12.59  16.62  23.21 

In [26]:
#print params_separate['log_like'].shape
loo, loos, ks = psisloo(params_separate['log_like'])
print loo
print loos
print ks

-161.923784276
[-30.96754925 -25.90462081 -26.55755364 -23.36716085 -25.66227433
 -29.46462538]
[ 1.96311373  1.61775755  1.63485495  1.56473963  1.63354363  1.76972595]
